In [ ]:
import datetime

In [ ]:
today = datetime.datetime.now().strftime('%Y%m%d')
hdf_name = f'trend_{today}.h5'
hdf_name

'trend_20211009.h5'

In [ ]:
!pip install pytrends numerapi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# %cd "/content/drive/My Drive/signals"
# import sys
# ntent/drive/My Drive/signals')

Mounted at /content/drive
/content/drive/My Drive/signals


In [ ]:
import numerapi
import pandas as pd
import time
from pytrends.request import TrendReq

In [ ]:
napi = numerapi.SignalsAPI()

In [ ]:
eligible_tickers = pd.Series(napi.ticker_universe(), name='numerai_ticker')

In [ ]:
ticker_map = pd.read_csv('https://numerai-signals-public-data.s3-us-west-2.amazonaws.com/signals_ticker_map_w_bbg.csv')

In [ ]:
trend_search_map = pd.read_csv('trend_search_list.csv')

In [ ]:
trend_search_map

,ticker,bloomberg_ticker,yahoo,Symbol,market,trendprefix,search_word,trend_topics
0,A,A US,A,A,NYSE,NYSE,NYSE:A,/m/07zkrqj
1,AA,AA US,AA,AA,NYSE,NYSE,NYSE:AA,/g/11c1wl07cj
2,AAL,AAL US,AAL,AAL,NASDAQ,NASDAQ,NASDAQ:AAL,/g/1ylhlpyzj
3,AAOI,AAOI US,AAOI,AAOI,NASDAQ,NASDAQ,NASDAQ:AAOI,/g/12vdghfvs
4,AAON,AAON US,AAON,AAON,NASDAQ,NASDAQ,NASDAQ:AAON,/m/07zljw8
...,...,...,...,...,...,...,...,...
2480,9974 JT,9974 JP,9974.T,9974,TYO,TYO,TYO:9974,/m/0cktqy5
2481,9983 JT,9983 JP,9983.T,9983,TYO,TYO,TYO:9983,/m/0ck_dyr
2482,9984 JT,9984 JP,9984.T,9984,TYO,TYO,TYO:9984,/m/0ckn0z1
2483,9987 JT,9987 JP,9987.T,9987,TYO,TYO,TYO:9987,/m/0ckhlm7


In [ ]:
ticker_map

,ticker,bloomberg_ticker,yahoo
0,000060 KS,000060 KS,000060.KS
1,000080 KS,000080 KS,000080.KS
2,000100 KS,000100 KS,000100.KS
3,000120 KS,000120 KS,000120.KS
4,000210 KS,000210 KS,000210.KS
...,...,...,...
5335,ZUO,ZUO US,ZUO
5336,ZURN SW,ZURN SW,ZURN.SW
5337,ZWS,ZWS US,ZWS
5338,ZYXI,ZYXI US,ZYXI


In [ ]:
trend_search_map = pd.merge(ticker_map, trend_search_map, on='ticker')

In [ ]:
trend_search_map

,ticker,bloomberg_ticker_x,yahoo_x,bloomberg_ticker_y,yahoo_y,Symbol,market,trendprefix,search_word,trend_topics
0,1332 JT,1332 JP,1332.T,1332 JP,1332.T,1332,TYO,TYO,TYO:1332,/m/0ckxfwv
1,1407 JT,1407 JP,1407.T,1407 JP,1407.T,1407,TYO,TYO,TYO:1407,/g/12m1zr_9z
2,1605 JT,1605 JP,1605.T,1605 JP,1605.T,1605,TYO,TYO,TYO:1605,/m/0ck_hjc
3,1662 JT,1662 JP,1662.T,1662 JP,1662.T,1662,TYO,TYO,TYO:1662,/m/0ckbwbw
4,1719 JT,1719 JP,1719.T,1719 JP,1719.T,1719,TYO,TYO,TYO:1719,/m/0cksqmw
...,...,...,...,...,...,...,...,...,...,...
2311,ZNTL,ZNTL US,ZNTL,ZNTL US,ZNTL,ZNTL,NASDAQ,NASDAQ,NASDAQ:ZNTL,/g/11ftcsl8sk
2312,ZS,ZS US,ZS,ZS US,ZS,ZS,NASDAQ,NASDAQ,NASDAQ:ZS,/g/11gglps7n0
2313,ZTS,ZTS US,ZTS,ZTS US,ZTS,ZTS,NYSE,NYSE,NYSE:ZTS,/g/1203m0r5b
2314,ZUMZ,ZUMZ US,ZUMZ,ZUMZ US,ZUMZ,ZUMZ,NASDAQ,NASDAQ,NASDAQ:ZUMZ,/m/07zkxzq


In [ ]:
trend_dfs = []

In [ ]:
for x in range(len(trend_search_map)):
    try:
        df = pd.read_hdf(hdf_name, f'trenddata_{x}')
        trend_dfs.append(df)
    except:
        break

In [ ]:
ticker2bloomberg = {}
for x in ticker_map.itertuples():
    # print(x.ticker, x.bloomberg_ticker)
    ticker2bloomberg[x.ticker] = x.bloomberg_ticker

In [ ]:
def save_trends():
    for x in range(len(trend_dfs)):
        df = trend_dfs[x]
        print(x, df.columns[0])
        df.to_hdf(hdf_name, f'trenddata_{x}')

In [ ]:
pytrends = TrendReq(hl='en-US', tz=360, timeout=(10,25), retries=2, backoff_factor=0.1) # requests_args={'verify':False}
# pytrends = TrendReq(hl='en-US', tz=360, timeout=(10,25), retries=10, backoff_factor=1) # requests_args={'verify':False}

have_map = {}
for df in trend_dfs:
    have_map[df.columns[0]] = True

for x in trend_search_map.itertuples():
#     print(x)
    if x.search_word in ['NASDAQ:CZR']:
        print(f'skip {x.search_word}')
        continue
    if have_map.get(x.ticker) is None:
        ok = False
        print(x.search_word, x.trend_topics)
#           print(x)
        for trycount in range(5):
            try:
                kw_list = [x.trend_topics]
                time.sleep(0.3)
                #time.sleep(3)
                pytrends.build_payload(kw_list=kw_list, timeframe='today 12-m', geo='', gprop='', cat=0)
                z = pytrends.interest_over_time()
                if len(z) > 0:
                    # print(z)
                    z = z[x.trend_topics]
                    z = z.rename(x.ticker)
                    z = z.to_frame()
                    trend_dfs.append(z)
                ok = True
                break
            except Exception as e:
                print(e)
        if not ok:
            continue
            #save_trends()
            #gotofail()
#     break
trend_dfs[-1]

ストリーミング出力は最後の 5000 行に切り捨てられました。
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
NYSE:LHX /m/07zkq2s
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
NYSE:LII /m/07zkr3s
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
NASDAQ:LILAK /g/11c3kq2x58
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google return

,CORT
date,
2020-10-11,25
2020-10-18,50
2020-10-25,0
2020-11-01,23
2020-11-08,49
2020-11-15,24
2020-11-22,24
2020-11-29,0
2020-12-06,48


In [ ]:
save_trends()

2021-10-09 19:04:30,694 INFO numexpr.utils: Note: NumExpr detected 40 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2021-10-09 19:04:30,695 INFO numexpr.utils: NumExpr defaulting to 8 threads.


0 1332 JT
1 1407 JT
2 1605 JT
3 1662 JT
4 1719 JT
5 1721 JT
6 1801 JT
7 1802 JT
8 1803 JT
9 1808 JT
10 1812 JT
11 1820 JT
12 1878 JT
13 1893 JT
14 1925 JT
15 1928 JT
16 1951 JT
17 1973 JT
18 2120 JT
19 2121 JT
20 2124 JT
21 2127 JT
22 2130 JT
23 2146 JT
24 2154 JT
25 2175 JT
26 2181 JT
27 2201 JT
28 2206 JT
29 2212 JT
30 2229 JT
31 2264 JT
32 2267 JT
33 2269 JT
34 2270 JT
35 2281 JT
36 2282 JT
37 2296 JT
38 2327 JT
39 2331 JT
40 2337 JT
41 2371 JT
42 2379 JT
43 2384 JT
44 2412 JT
45 2413 JT
46 2427 JT
47 2432 JT
48 2471 JT
49 2491 JT
50 2501 JT
51 2502 JT
52 2503 JT
53 2579 JT
54 2587 JT
55 2593 JT
56 2607 JT
57 2651 JT
58 2678 JT
59 2685 JT
60 2695 JT
61 2702 JT
62 2768 JT
63 2782 JT
64 2784 JT
65 2801 JT
66 2802 JT
67 2809 JT
68 2810 JT
69 2815 JT
70 2871 JT
71 2875 JT
72 2897 JT
73 2914 JT
74 2930 JT
75 3003 JT
76 3031 JT
77 3038 JT
78 3048 JT
79 3050 JT
80 3064 JT
81 3086 JT
82 3088 JT
83 3092 JT
84 3097 JT
85 3099 JT
86 3101 JT
87 3110 JT
88 3141 JT
89 3186 JT
90 3197 JT
91 3231 J

In [ ]:
signals_dfs = []
for df in trend_dfs:
    ticker = df.columns[0]
    print(ticker)
    df = df.rename(columns={ticker: 'trend'})
    df = df.replace(0, 1)
    df['trend_ave13'] = df.rolling(13).mean()
    df['trend_kairi13'] = (df['trend'] - df['trend_ave13']) / df['trend_ave13']
    df['ticker'] = ticker
    df['numerai_ticker'] = ticker2bloomberg[ticker]
    df['friday_date'] = df.index + datetime.timedelta(days=5)
    today = df.tail(1)['friday_date'][0]
    df['data_type'] = df['friday_date'].apply(lambda x : 'live' if x == today else 'validation')
    signals_dfs.append(df)
# df

1332 JT
1407 JT
1605 JT
1662 JT
1719 JT
1721 JT
1801 JT
1802 JT
1803 JT
1808 JT
1812 JT
1820 JT
1878 JT
1893 JT
1925 JT
1928 JT
1951 JT
1973 JT
2120 JT
2121 JT
2124 JT
2127 JT
2130 JT
2146 JT
2154 JT
2175 JT
2181 JT
2201 JT
2206 JT
2212 JT
2229 JT
2264 JT
2267 JT
2269 JT
2270 JT
2281 JT
2282 JT
2296 JT
2327 JT
2331 JT
2337 JT
2371 JT
2379 JT
2384 JT
2412 JT
2413 JT
2427 JT
2432 JT
2471 JT
2491 JT
2501 JT
2502 JT
2503 JT
2579 JT
2587 JT
2593 JT
2607 JT
2651 JT
2678 JT
2685 JT
2695 JT
2702 JT
2768 JT
2782 JT
2784 JT
2801 JT
2802 JT
2809 JT
2810 JT
2815 JT
2871 JT
2875 JT
2897 JT
2914 JT
2930 JT
3003 JT
3031 JT
3038 JT
3048 JT
3050 JT
3064 JT
3086 JT
3088 JT
3092 JT
3097 JT
3099 JT
3101 JT
3110 JT
3141 JT
3186 JT
3197 JT
3231 JT
3288 JT
3289 JT
3291 JT
3341 JT
3349 JT
3360 JT
3382 JT
3391 JT
3395 JT
3397 JT
3401 JT
3402 JT
3405 JT
3407 JT
3436 JT
3479 JT
3543 JT
3549 JT
3563 JT
3626 JT
3632 JT
3635 JT
3655 JT
3659 JT
3668 JT
3678 JT
3738 JT
3762 JT
3765 JT
3769 JT
3774 JT
3861 JT
3863 JT


In [ ]:
signal_df = pd.concat(signals_dfs)
signal_df = signal_df.dropna()
signal_df = signal_df.sort_index()
signal_df

,trend,trend_ave13,trend_kairi13,ticker,numerai_ticker,friday_date,data_type
date,,,,,,,
2021-01-03,1,33.538462,-0.970183,1332 JT,1332 JP,2021-01-08,validation
2021-01-03,25,16.384615,0.525822,ATH,ATH US,2021-01-08,validation
2021-01-03,1,15.846154,-0.936893,3341 JT,3341 JP,2021-01-08,validation
2021-01-03,20,26.769231,-0.252874,ATGE,ATGE US,2021-01-08,validation
2021-01-03,1,17.923077,-0.944206,3349 JT,3349 JP,2021-01-08,validation
...,...,...,...,...,...,...,...
2021-10-03,1,33.307692,-0.969977,7181 JT,7181 JP,2021-10-08,live
2021-10-03,72,37.076923,0.941909,9143 JT,9143 JP,2021-10-08,live
2021-10-03,21,24.153846,-0.130573,4480 JT,4480 JP,2021-10-08,live


In [ ]:
signal_df = signal_df.reset_index()

In [ ]:
def make_rank(pred):
    return pred.rank(pct=True, method="first")
signal_df['signal'] = signal_df.groupby('date')['trend_kairi13'].apply(make_rank)
signal_df

,date,trend,trend_ave13,trend_kairi13,ticker,numerai_ticker,friday_date,data_type,signal
0,2021-01-03,1,33.538462,-0.970183,1332 JT,1332 JP,2021-01-08,validation,0.014519
1,2021-01-03,25,16.384615,0.525822,ATH,ATH US,2021-01-08,validation,0.764065
2,2021-01-03,1,15.846154,-0.936893,3341 JT,3341 JP,2021-01-08,validation,0.205082
3,2021-01-03,20,26.769231,-0.252874,ATGE,ATGE US,2021-01-08,validation,0.437387
4,2021-01-03,1,17.923077,-0.944206,3349 JT,3349 JP,2021-01-08,validation,0.163339
...,...,...,...,...,...,...,...,...,...
22035,2021-10-03,1,33.307692,-0.969977,7181 JT,7181 JP,2021-10-08,live,0.030853
22036,2021-10-03,72,37.076923,0.941909,9143 JT,9143 JP,2021-10-08,live,0.838475
22037,2021-10-03,21,24.153846,-0.130573,4480 JT,4480 JP,2021-10-08,live,0.522686
22038,2021-10-03,1,28.846154,-0.965333,3050 JT,3050 JP,2021-10-08,live,0.079855


In [ ]:
signal_df['friday_date'] = signal_df['friday_date'].dt.strftime('%Y%m%d')

In [ ]:
signal_df[['numerai_ticker', 'friday_date', 'data_type', 'signal']]

,numerai_ticker,friday_date,data_type,signal
0,1332 JP,20210108,validation,0.014519
1,ATH US,20210108,validation,0.764065
2,3341 JP,20210108,validation,0.205082
3,ATGE US,20210108,validation,0.437387
4,3349 JP,20210108,validation,0.163339
...,...,...,...,...
22035,7181 JP,20211008,live,0.030853
22036,9143 JP,20211008,live,0.838475
22037,4480 JP,20211008,live,0.522686
22038,3050 JP,20211008,live,0.079855


In [ ]:
signal_df[['numerai_ticker', 'friday_date', 'data_type', 'signal']].to_csv('trend_signal_upload.csv', index=False)

In [ ]:
public_id = "ZRGZYLZAQVG6QX3OYKBFRNCUYUZECI6B"
secret_key = "YLCNLZURRAY3BF5THJBMDUPYZC2L3D6LU7ZT2ERZCW3CRDCOBQS3IQELVY4OBCX2"
model_id = '197fef41-d356-4efa-a8de-5da067589319'
napi = numerapi.SignalsAPI(public_id=public_id, secret_key=secret_key)
napi.upload_predictions('trend_signal_upload.csv', model_id=model_id)

2021-10-09 19:04:48,646 INFO numerapi.base_api: uploading predictions...


'46766474-0e9b-435f-9c06-6c9d3e68f79d'